In [7]:
from tensorflow.keras.applications import VGG16, ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Input, Flatten, Dense, Lambda
from sklearn.model_selection import train_test_split
import numpy as np

from functions import prepare_data, euclidean_distance, contrastiveLoss

data_directory = 'sketch_small'
TARGET_SIZE = (224, 224, 3)
EPOCHS = 10
BATCH_SIZE = 32


In [6]:
# Load pretrained VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Remove top classification layers
base_model = Model(inputs=base_model.input, outputs=base_model.get_layer('block5_pool').output)

ImportError: Loading a H5 file requires `h5py` to be installed.

In [ ]:
input_shape = TARGET_SIZE
input_left = Input(shape=input_shape)
input_right = Input(shape=input_shape)

embedding_left = base_model(input_left)
embedding_right = base_model(input_right)

# Compute L1 distance between embeddings
L1_distance = Lambda(lambda tensors: abs(tensors[0] - tensors[1]))([embedding_left, embedding_right])

# Add dense layer for prediction
dense = Dense(1, activation='sigmoid')(L1_distance)

# Connect inputs and outputs to create Siamese model
siamese_model = Model(inputs=[input_left, input_right], outputs=dense)

In [ ]:
#1
def siamese_network(input_shape=TARGET_SIZE):
    input_left = Input(shape=input_shape)
    input_right = Input(shape=input_shape)
    
    base_branch = base_model
    
    encoded_left = base_branch(input_left)
    encoded_right = base_branch(input_right)
    
    L1_distance = Lambda(lambda tensors: abs(tensors[0] - tensors[1]))([encoded_left, encoded_right])
    
    prediction = Dense(1, activation='sigmoid')(L1_distance)
    
    siamese_network = Model(inputs=[input_left, input_right], outputs=prediction)
    
    return siamese_network

siamese_model = siamese_network()
siamese_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

siamese_model.fit([X_train_left, X_train_right], y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_split=0.2)

In [ ]:
X_train, X_test, y_train, y_test = prepare_data(data_directory, TARGET_SIZE)  
X_train_left = X_train[:, 0]
X_train_right = X_train[:, 1]

In [ ]:
print(X_train.shape, X_train_left.shape, y_train.shape)

In [ ]:
siamese_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train Siamese network
siamese_model.fit([X_train_left, X_train_right], y_train, epochs=10, batch_size=32, validation_split=0.2)
